In [145]:
from llama_index import SimpleDirectoryReader

In [146]:
pdf_path = "../../data/apple/AAPL.pdf"

In [147]:
import fitz  # Import PyMuPDF
from markdownify import markdownify as md


def convert_pdf_to_html(pdf_path):
    doc = fitz.open(pdf_path)
    html_content = ""

    for page_num in range(3, len(doc)):
        page = doc.load_page(page_num)
        html_content += page.get_text("html")

    return html_content

def html_to_markdown(html_content):
    return md(html_content)

# Example usage
html_content = convert_pdf_to_html(pdf_path)
markdown_text = html_to_markdown(html_content)
print(markdown_text)




*This Annual Report on Form 10-K (“Form 10-K”) contains forward-looking statements, within the meaning of the Private* 


*Securities Litigation Reform Act of 1995, that involve risks and uncertainties. Many of the forward-looking statements are* 


*located in Part I, Item 1 of this Form 10-K under the heading “Business” and Part II, Item 7 of this Form 10-K under the heading* 


*“Management’s Discussion and Analysis of Financial Condition and Results of Operations.” Forward-looking statements* 


*provide current expectations of future events based on certain assumptions and include any statement that does not directly* 


*relate to any historical or current fact. For example, statements in this Form 10-K regarding the potential future impact of the* 


*COVID-19 pandemic on the Company’s business and results of operations are forward-looking statements. Forward-looking* 


*statements can also be identified by words such as “future,” “anticipates,” “believes,” “estimates,” “expec

In [148]:
# reformat bullet points
markdown_text = markdown_text.replace("• \n\n\n", "- ")
markdown_text = markdown_text.replace("\n\n", "")

In [149]:
items = markdown_text.split("**Item ")
items = items[1:]
len(items)  

22

In [150]:
for item in items:
    print(item)
    print("-"*30)

1.** 
**Business**
**Company Background**
The Company designs, manufactures and markets smartphones, personal computers, tablets, wearables and accessories, and 
sells a variety of related services. The Company’s fiscal year is the 52- or 53-week period that ends on the last Saturday of 
September.
**Products**
*iPhone*
iPhone® is the Company’s line of smartphones based on its iOS operating system. The iPhone line includes iPhone 14 Pro, 
iPhone 14, iPhone 13, iPhone SE®, iPhone 12 and iPhone 11.
*Mac*
Mac® is the Company’s line of personal computers based on its macOS® operating system. The Mac line includes laptops 
MacBook Air® and MacBook Pro®, as well as desktops iMac®, Mac mini®, Mac Studio™ and Mac Pro®.
*iPad*
iPad® is the Company’s line of multipurpose tablets based on its iPadOS® operating system. The iPad line includes iPad Pro®, 
iPad Air®, iPad and iPad mini®.
*Wearables, Home and Accessories*
Wearables, Home and Accessories includes:
•
AirPods®, the Company’s wireless hea

In [154]:
import re
pattern = r"\*\*I.*\*\* "
item_titles = re.findall(pattern, markdown_text)
print(len(item_titles))

item_titles = [item.replace("**", "").strip() for item in item_titles]

# Print the headers
for item_title in item_titles:
    print(item_title)

23
Item 1.
Item 1A.
Investment in new business strategies and acquisitions could disrupt the Company’s ongoing business, present risks not*
Item 1B.
Item 2.
Item 3.
Item 4.
Item 5.
Item 6.
Item 7.
Item 7A.
Item 8.
Item 9.
Item 9A.
Item 9B.
Item 9C.
Item 10.
Item 11.
Item 12.
Item 13.
Item 14.
Item 15.
Item 16.


In [152]:
dw cer

SyntaxError: invalid syntax (461090463.py, line 1)

In [ ]:
# contents = []

# for i, item in enumerate(items):
#     c = {}
#     c['item_title'] = item_titles[i]
#     c['content'] = item
#     contents.append(c)

In [ ]:
# contents

In [ ]:
with open("markdown.md", "w") as f:
    f.write(markdown_text)

In [ ]:
import re
all_titles = []


def get_section_titles(item):
    titles = re.findall(r'\*\*(.*?)\*\*', item)
    titles = [title for title in titles if re.search('[a-zA-Z]', title)]
    return titles
    

In [ ]:
def extract_content_for_title(start_marker, end_marker, text):
    pattern = f"{re.escape(start_marker)}(.*?){re.escape(end_marker)}"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        extracted_text = match.group(1)
    else:
        print("No text found between the markers.")
        return None

    return extracted_text


In [ ]:
from llama_index.text_splitter import SentenceSplitter

all_data = []
chunk_size = 1000

splitter = SentenceSplitter(
    chunk_size=chunk_size,
    chunk_overlap=100
)

for i, item in enumerate(items):
    titles = get_section_titles(item)
    for j, title in enumerate(titles[:-1]):

        content = extract_content_for_title(title, titles[j+1], item)

        if len(content.split()) > chunk_size:
            chunks = splitter.split_text(content)
            for chunk in chunks:
                c = {}
                c['item_title'] = item_titles[i]
                c['section_title'] = title
                c['content'] = chunk
                all_data.append(c)

        # elif len(content.split()) < 20:
        #     continue 
        
        else:
            c = {}
            c['item_title'] = item_titles[i]
            c['section_title'] = title
            c['content'] = content
            all_data.append(c)

In [ ]:
all_data

[]

1. What is the ideal max chunk size?

In [ ]:
# import matplotlib.pyplot as plt


# char_counts = [len(entry['content']) for entry in all_data]
# word_counts = [len(entry['content'].split()) for entry in all_data]

# # Plotting
# plt.figure(figsize=(14, 7))

# # Plot for character counts
# plt.subplot(1, 2, 1)
# plt.bar(range(len(all_data)), char_counts, color='skyblue')
# plt.title('Character Count in Content')
# plt.xlabel('JSON Object Index')
# plt.ylabel('Character Count')
# plt.xticks(range(len(all_data)))

# # Plot for word counts
# plt.subplot(1, 2, 2)
# plt.bar(range(len(all_data)), word_counts, color='lightgreen')
# plt.title('Word Count in Content')
# plt.xlabel('JSON Object Index')
# plt.ylabel('Word Count')
# plt.xticks(range(len(all_data)))

# plt.tight_layout()  # Adjust the layout so that plots fit nicely
# # plt.show()  # This line is commented out since you requested the code, not the plot itself


In [ ]:
import statistics
import pandas as pd

chunk_length = []

for content in all_data:
    chunk_length.append(len(content['content'].split()))

s = pd.Series(chunk_length)
s.describe()


count       0
unique      0
top       NaN
freq      NaN
dtype: object

In [ ]:
for content in all_data:
    print(f"ITEM TITLE: {content['item_title']}")
    print(f"SECTION TITLE: {content['section_title']}")
    print(f"CONTENT: {content['content']}")
    print("-"*50)

In [ ]:
count = 0

for i, content in enumerate(all_data[:-1]):
    if len(content['content'].split()) <= 50:
        print(content['item_title'])
        
        print(content['section_title'])
        print(all_data[i+1]['section_title'])
        print(content['content'])
        count += 1
        print("-"*50)

In [ ]:
count

0

In [ ]:
len(all_data)

0

In [ ]:
import json

with open("data/contents.json", "w") as f:
    json.dump(all_data, f, indent=4)